# Extract APE (Anatomical Positional Embedding) for NSCLC

### Recommended: Run APE in a Separate Docker Container
To avoid potential conflicts with the RadiomicsRetrieval environment, it is recommended to run APE extraction in a separate container:

```bash
docker run --shm-size=192gb --gpus=all -it \
  -v /path/to/your/project/:/workspace \
  --name ape \
  pytorch/pytorch:2.3.1-cuda12.1-cudnn8-devel /bin/bash
```

### Inside the container: Install APE
```bash
git clone https://github.com/mishgon/ape.git && cd ape && pip install -e .
```

In [ ]:
import os
import numpy as np
import SimpleITK as sitk
import torch
from torch import nn
import torch.nn.functional as F
from typing import Optional, Union, Tuple, Literal, Sequence
import itertools
import math
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from ape.model import APE

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Device", device)

model = APE(pretrained=True).eval().to(device)

data_root = '/workspace/RadiomicsRetrieval/data/NSCLC'
images_root = os.path.join(data_root, 'images')
apes_npy_root = os.path.join(data_root, 'apes_npy')
apes_nii_root = os.path.join(data_root, 'apes_nii')
os.makedirs(apes_npy_root, exist_ok=True)
os.makedirs(apes_nii_root, exist_ok=True)

images = os.listdir(images_root)
for image_name in tqdm(images):
    image_path = os.path.join(images_root, image_name)

    image_sitk = sitk.ReadImage(image_path)
    image_np = sitk.GetArrayFromImage(image_sitk)
    image_np = np.transpose(image_np, (2, 1, 0))
    
    image_np = (image_np - image_np.mean()) / image_np.std()
    image_tensor = torch.tensor(image_np).unsqueeze(0).unsqueeze(0).float().to(device)

    with torch.no_grad():
        ape_tensor = model(image_tensor).cpu()
        stride = 4
        ape_tensor = F.interpolate(ape_tensor, size=image_np.shape, mode='trilinear', align_corners=False).squeeze(0)

    ape_npy_path = os.path.join(apes_npy_root, image_name.replace('.nii.gz', '.npy'))
    np.save(ape_npy_path, ape_tensor.numpy())

    for i in range(3):
        ape_nii_path = os.path.join(apes_nii_root, f'{image_name.replace(".nii.gz", "")}_ape_{i}.nii.gz')
        ape_np = ape_tensor[i].numpy()
        ape_np = np.transpose(ape_np, (2, 1, 0))
        ape_sitk = sitk.GetImageFromArray(ape_np)
        ape_sitk.SetSpacing(image_sitk.GetSpacing())
        ape_sitk.SetOrigin(image_sitk.GetOrigin())
        ape_sitk.SetDirection(image_sitk.GetDirection())
        sitk.WriteImage(ape_sitk, ape_nii_path)
